## Sarsa利用tkinter实现（包含Sarsa($\lambda$)）
### Sarsa与QLearning的对比
初始化Q(s,a)表格，其中s是状态，a是动作<br>
Repeat (for each eposode):<br>
&#160;&#160;&#160;&#160; 初始化 s<br>
&#160;&#160;&#160;&#160; Repeat (for each step of eposode):<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 从Q表格中根据当前状态s选择动作a（在$\epsilon = 0.9$ 的概率下，否则随机选择动作）<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 根据动作a，得到当前的观察r和状态s'<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; $Q(s,a) = Q(s,a) + \alpha [r + \gamma Q(s',a')-Q(s,a)]$（Sarsa属于实践派，与QLearning的$Q(s,a) = Q(s,a) + \alpha [r + \gamma max_{a'}Q(s',a')-Q(s,a)]$不同）<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; s = s'<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 直到s是终点，结束循环<br>

处于状态 s 时，根据当前 Q表格以及一定的策略来选取动作a，进而观测到下一步状态s'，并再次根据当前Q表格及相同的策略选择动作a'，这样就有了一个$[s，a，r，s'，a']$序列。处于状态 s' 时，Sarsa知道了要采取哪个a'，并真的采取了这个动作。动作a的选取遵循e-greedy 策略，目标Q值的计算也是根据策略得到的动作a'计算得来。<br>

而对QLearning而言，处于状态 s' 时，仅计算了 在 s' 时要采取哪个 a' 可以得到更大的 Q 值，并没有真的采取这个动作 a'；动作 a 的选取是根据当前 Q 网络以及策略（e-greedy），目标 Q 值的计算是根据 Q 值最大的动作 a' 计算得来。


### 强化学习的大脑
分别写了Sarsa和QLearning两个类，并继承了一些基础的功能

In [1]:
import numpy as np
import pandas as pd


class RL(object):
    def __init__(self, action_space, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = action_space  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy

        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.rand() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose on in these actions
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, *args):
        pass


# off-policy 离线学习
class QLearningTable(RL):
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        super(QLearningTable, self).__init__(actions, learning_rate, reward_decay, e_greedy)

    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update


# on-policy 在线学习
class SarsaTable(RL):

    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        super(SarsaTable, self).__init__(actions, learning_rate, reward_decay, e_greedy)

    def learn(self, s, a, r, s_, a_): # Sarsa和Qlearning最大的不同，需要考虑下一步的行动a_
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, a_]  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update

### 利用tkinter建立环境
同上一节

In [2]:
import numpy as np
import time
import sys
if sys.version_info.major == 2:
    import Tkinter as tk
else:
    import tkinter as tk


UNIT = 40   # pixels
MAZE_H = 4  # grid height
MAZE_W = 4  # grid width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('maze')
        self.geometry('{0}x{1}'.format(MAZE_H * UNIT, MAZE_H * UNIT))
        self._build_maze()

    def _build_maze(self):
        self.canvas = tk.Canvas(self, bg='white',
                           height=MAZE_H * UNIT,
                           width=MAZE_W * UNIT)

        # create grids
        for c in range(0, MAZE_W * UNIT, UNIT):
            x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, MAZE_H * UNIT, UNIT):
            x0, y0, x1, y1 = 0, r, MAZE_W * UNIT, r
            self.canvas.create_line(x0, y0, x1, y1)

        # create origin
        origin = np.array([20, 20])

        # hell
        hell1_center = origin + np.array([UNIT * 2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0] - 15, hell1_center[1] - 15,
            hell1_center[0] + 15, hell1_center[1] + 15,
            fill='black')
        # hell
        hell2_center = origin + np.array([UNIT, UNIT * 2])
        self.hell2 = self.canvas.create_rectangle(
            hell2_center[0] - 15, hell2_center[1] - 15,
            hell2_center[0] + 15, hell2_center[1] + 15,
            fill='black')

        # create oval
        oval_center = origin + UNIT * 2
        self.oval = self.canvas.create_oval(
            oval_center[0] - 15, oval_center[1] - 15,
            oval_center[0] + 15, oval_center[1] + 15,
            fill='yellow')

        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')

        # pack all
        self.canvas.pack()

    def reset(self):
        self.update()
        time.sleep(0.5)
        self.canvas.delete(self.rect)
        origin = np.array([20, 20])
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')
        # return observation
        return self.canvas.coords(self.rect)

    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0, 0])
        if action == 0:   # up
            if s[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:   # down
            if s[1] < (MAZE_H - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:   # right
            if s[0] < (MAZE_W - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 3:   # left
            if s[0] > UNIT:
                base_action[0] -= UNIT

        self.canvas.move(self.rect, base_action[0], base_action[1])  # move agent

        s_ = self.canvas.coords(self.rect)  # next state

        # reward function
        if s_ == self.canvas.coords(self.oval):
            reward = 1
            done = True
            s_ = 'terminal'
        elif s_ in [self.canvas.coords(self.hell1), self.canvas.coords(self.hell2)]:
            reward = -1
            done = True
            s_ = 'terminal'
        else:
            reward = 0
            done = False

        return s_, reward, done

    def render(self):
        time.sleep(0.1)
        self.update()

### 主函数
* 更新环境
* 选择行动并得到状态
* （新）选择下一步的动作
* 选择行动并得到下一步的状态和奖赏（这一步需要带入上一步得到的动作action_）
* 学习并更新Q表格
* 更新状态

In [3]:
def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()

        # RL choose action based on observation
        action = RL.choose_action(str(observation))

        while True:
            # fresh env
            env.render()

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL choose action based on next observation
            action_ = RL.choose_action(str(observation_))

            # RL learn from this transition (s, a, r, s, a) ==> Sarsa
            RL.learn(str(observation), action, reward, str(observation_), action_)

            # swap observation and action
            observation = observation_
            action = action_

            # break while loop when end of this episode
            if done:
                break

    # end of game
    print('game over')
    env.destroy()

In [ ]:
env = Maze()
RL = SarsaTable(actions=list(range(env.n_actions)))

env.after(100, update)
env.mainloop()

比起QLearning，可以看出Sarsa更加保守，尝试过程中尽量不贴近陷阱的地带。
![](picture3.gif)

## Sarsa($\lambda$)
Sarsa($\lambda$)算法是Sarsa 的改进版，二者的主要区别在于：

在每次选择了动作并获得reward后，Sarsa只对前一步$Q(s,a)$进行更新，Sarsa($\lambda$) 则会对获得reward之前的所有步进行更新。<br>

Sarsa(lambda)算法的流程如下：

![](picture4.png)

从上图可以看出，和Sarsa相比，Sarsa($\lambda$)算法中多了一个矩阵E (eligibility trace)，它是用来保存在路径中所经历的每一步，因此在每次更新时也会对之前经历的步进行更新。<br>

参数lambda取值范围为[0, 1] ，如果 lambda = 0，Sarsa($\lambda$) 将退化为Sarsa，即只更新获取到 reward 前经历的最后一步；如果 lambda = 1，Sarsa($\lambda$) 更新的是获取到 reward 前的所有步。$\lambda$可理解为脚步的衰变值，即离奶酪越近的步越重要，越远的步则对于获取奶酪不是太重要。<br>

和Sarsa相比，Sarsa($\lambda$)算法有如下优势：<br>

* Sarsa虽然会边走边更新，但是在没有到达terminate之前，当前步的Q值是没有任何变化的，直到达到terminate后，才会对前一步更新，而之前为了获取奖赏所走的所有步都被认为和获取奖赏没关系。Sarsa($\lambda$)则会对获取奖赏所走的步都进行更新，离奖赏越近的步越重要，越远的则越不重要（由参数$\lambda$控制衰减幅度）。因此，Sarsa($\lambda$) 能够更加快速有效的学到最优的policy。

* 在算法前几回合，探索者由于没有头绪, 可能在原地打转了很久，从而形成一些重复的环路，而这些环路对于算法的学习没有太大必要。Sarsa($\lambda$)则可解决该问题，具体做法是：在$E(s,a)←E(s,a)+1$这一步之前，可先令E(s)=0，即把状态s对应的行置为0，这样就只保留了最近一次到达状态s时所做的action。（下图的第三行）


![](picture5.png)

In [6]:
import numpy as np
import pandas as pd


class RL(object):
    def __init__(self, action_space, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = action_space  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy

        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.rand() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose on in these actions
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, *args):
        pass


# backward eligibility traces
class SarsaLambdaTable(RL):
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9, trace_decay=0.9):
        super(SarsaLambdaTable, self).__init__(actions, learning_rate, reward_decay, e_greedy)

        # backward view, eligibility trace.
        self.lambda_ = trace_decay # lamda的衰减值
        self.eligibility_trace = self.q_table.copy() # E矩阵记录状态中动作出现过的次数

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q table
            to_be_append = pd.Series(
                    [0] * len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            self.q_table = self.q_table.append(to_be_append)

            # also update eligibility trace！！！
            self.eligibility_trace = self.eligibility_trace.append(to_be_append)

    def learn(self, s, a, r, s_, a_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, a_]  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        error = q_target - q_predict

        # increase trace amount for visited state-action pair

        # Method 1:（上图的第二行）
        # self.eligibility_trace.loc[s, a] += 1

        # Method 2:（上图的第三行）
        self.eligibility_trace.loc[s, :] *= 0
        self.eligibility_trace.loc[s, a] = 1

        # Q update
        self.q_table += self.lr * error * self.eligibility_trace

        # decay eligibility trace after update
        self.eligibility_trace *= self.gamma*self.lambda_

In [ ]:
env = Maze()
RL = SarsaLambdaTable(actions=list(range(env.n_actions)))

env.after(100, update)
env.mainloop()

![](picture6.gif)